## Create a Calculator Tool

In [1]:
import rigging as rg
import functools
from functools import cached_property
import asyncio
import typing as t

class Calculator:
    def __init__(self):
        self.result = 0

    @cached_property
    def tools(self) -> list[t.Callable[..., t.Any]]:
        def wrap(func: t.Callable[..., t.Any]) -> t.Callable[..., t.Any]:
            @rg.tool(catch=True)
            @functools.wraps(func)
            async def wrapper(*args: t.Any, **kwargs: t.Any) -> t.Any:
                return await asyncio.to_thread(func, *args, **kwargs)

            return wrapper

        return [
            wrap(func)
            for func in (
                self.add,
                self.subtract,
                self.multiply,
                self.divide
            )
        ]

    def add(self, x: int, y: int) -> int:
        """Adds two integers."""
        return x + y

    def subtract(self, x: int, y: int) -> int:
        """Subtracts the second integer from the first."""
        return x - y

    def multiply(self, x: int, y: int) -> int:
        """Multiplies two integers."""
        return x * y

    def divide(self, x: int, y: int) -> float:
        """Divides the first integer by the second."""
        if y == 0:
            raise ValueError("Cannot divide by zero.")
        return x / y


## Create a check answer callback

In [2]:
def check_answer(correct_answer: str) -> bool:
    """Check if the user's answer is correct."""
    async def evaluate(chat: rg.Chat) -> rg.Chat:
        if correct_answer in chat.last.content:
            chat.meta(result = {"answer": correct_answer, "user": chat.last.content, "correct": True})
        
        return chat
    
    return evaluate

## Create an agent that uses the Calculator

In [3]:
generator = rg.get_generator("groq/meta-llama/llama-4-maverick-17b-128e-instruct")

calculator = Calculator()

agent = await generator.chat(
    [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is 2 + 2?"}
    ]).using(calculator.tools).then(check_answer("4")).run()

agent

Chat(uuid=UUID('cc8a7e7d-adc5-48a6-b739-25b7aaf9853f'), messages=[Message(role='system', tool_calls=None, tool_call_id=None, content='You are a helpful assistant.'), Message(role='user', tool_calls=None, tool_call_id=None, content='What is 2 + 2?'), Message(role='assistant', tool_calls=[ToolCall(id='ycb70ar94', type='function', function=FunctionCall(name='add', arguments='{"x":2,"y":2}'))], tool_call_id=None, content=''), Message(role='tool', tool_calls=None, tool_call_id='ycb70ar94', content='4')], generated=[Message(role='assistant', tool_calls=None, tool_call_id=None, content='The result of 2 + 2 is 4.')], metadata={'result': {'answer': '4', 'user': 'The result of 2 + 2 is 4.', 'correct': True}}, stop_reason='stop', failed=False)

## Tokenize the chat with a tokenizer

In [4]:
await agent.to_tokens('Qwen/Qwen2.5-1.5B-Instruct')

TokenizedChat(text='<|im_start|>system\nYou are a helpful assistant.\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{"type": "function", "function": {"name": "add", "description": "Adds two integers.", "parameters": {"additionalProperties": false, "properties": {"x": {"title": "X", "type": "integer"}, "y": {"title": "Y", "type": "integer"}}, "required": ["x", "y"], "type": "object"}}}\n{"type": "function", "function": {"name": "subtract", "description": "Subtracts the second integer from the first.", "parameters": {"additionalProperties": false, "properties": {"x": {"title": "X", "type": "integer"}, "y": {"title": "Y", "type": "integer"}}, "required": ["x", "y"], "type": "object"}}}\n{"type": "function", "function": {"name": "multiply", "description": "Multiplies two integers.", "parameters": {"additionalProperties": false, "properties": {"x": {"title": "X", "type": 